In [1]:
import os
import sys
sys.path.append(os.path.abspath("../Video-Swin-Transformer"))

In [2]:
# Change teh working directory to a location that the code prefers
os.chdir("../Video-Swin-Transformer")

In [3]:
import argparse
import copy
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist, set_random_seed
from mmcv.utils import get_git_hash

from mmaction import __version__
from mmaction.apis import train_model
from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.utils import collect_env, get_root_logger, register_module_hooks

In [4]:
# TODO import test functions from mmcv and delete them from mmaction2
try:
    from mmcv.engine import multi_gpu_test, single_gpu_test
except (ImportError, ModuleNotFoundError):
    warnings.warn(
        'DeprecationWarning: single_gpu_test, multi_gpu_test, '
        'collect_results_cpu, collect_results_gpu from mmaction2 will be '
        'deprecated. Please install mmcv through master branch.')
    from mmaction.apis import multi_gpu_test, single_gpu_test

In [5]:
# Setup the cofiguration and data file
config_file = '../configs/bsl_config.py'
check_point_file = '../configs/swin_tiny_patch244_window877_kinetics400_1k.pth'
# , "model.backbone.pretrained="+check_point_file
cmd_options = [config_file, "--cfg-options", "model.backbone.use_checkpoint=True"]

In [6]:
def parse_args(parse_options=None):
    parser = argparse.ArgumentParser(description='Train a recognizer')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--work-dir', help='the dir to save logs and models')
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    parser.add_argument(
        '--validate',
        action='store_true',
        help='whether to evaluate the checkpoint during training')
    parser.add_argument(
        '--test-last',
        action='store_true',
        help='whether to test the checkpoint after training')
    parser.add_argument(
        '--test-best',
        action='store_true',
        help=('whether to test the best checkpoint (if applicable) after '
              'training'))
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        default={},
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file. For example, '
        "'--cfg-options model.backbone.depth=18 model.backbone.with_cp=True'")
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    parser.add_argument('--local_rank', type=int, default=0)
    
    if parse_options is None: 
        args = parser.parse_args()
    else:
        args = parser.parse_args(parse_options)
        
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    return args

In [7]:
def main(arg_options):
    args = parse_args(arg_options)

    cfg = Config.fromfile(args.config)

    cfg.merge_from_dict(args.cfg_options)

    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True

    # work_dir is determined in this priority:
    # CLI > config file > default (base filename)
    if args.work_dir is not None:
        # update configs according to CLI args if args.work_dir is not None
        cfg.work_dir = args.work_dir
    elif cfg.get('work_dir', None) is None:
        # use config filename as default work_dir if cfg.work_dir is None
        cfg.work_dir = osp.join('./work_dirs',
                                osp.splitext(osp.basename(args.config))[0])
    if args.resume_from is not None:
        cfg.resume_from = args.resume_from
    if args.gpu_ids is not None:
        cfg.gpu_ids = args.gpu_ids
    else:
        cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

    # init distributed env first, since logger depends on the dist info.
    if args.launcher == 'none':
        distributed = False
    else:
        distributed = True
        init_dist(args.launcher, **cfg.dist_params)
        _, world_size = get_dist_info()
        cfg.gpu_ids = range(world_size)

    # The flag is used to determine whether it is omnisource training
    cfg.setdefault('omnisource', False)

    # The flag is used to register module's hooks
    cfg.setdefault('module_hooks', [])

    # create work_dir
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    # dump config
    cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
    # init logger before other steps
    timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
    logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

    # init the meta dict to record some important information such as
    # environment info and seed, which will be logged
    meta = dict()
    # log env info
    env_info_dict = collect_env()
    env_info = '\n'.join([f'{k}: {v}' for k, v in env_info_dict.items()])
    dash_line = '-' * 60 + '\n'
    logger.info('Environment info:\n' + dash_line + env_info + '\n' +
                dash_line)
    meta['env_info'] = env_info

    # log some basic info
    logger.info(f'Distributed training: {distributed}')
    logger.info(f'Config: {cfg.pretty_text}')

    # set random seeds
    if args.seed is not None:
        logger.info(f'Set random seed to {args.seed}, '
                    f'deterministic: {args.deterministic}')
        set_random_seed(args.seed, deterministic=args.deterministic)
    cfg.seed = args.seed
    meta['seed'] = args.seed
    meta['config_name'] = osp.basename(args.config)
    meta['work_dir'] = osp.basename(cfg.work_dir.rstrip('/\\'))

    model = build_model(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))

    if len(cfg.module_hooks) > 0:
        register_module_hooks(model, cfg.module_hooks)

    if cfg.omnisource:
        # If omnisource flag is set, cfg.data.train should be a list
        assert isinstance(cfg.data.train, list)
        datasets = [build_dataset(dataset) for dataset in cfg.data.train]
    else:
        datasets = [build_dataset(cfg.data.train)]

    if len(cfg.workflow) == 2:
        # For simplicity, omnisource is not compatiable with val workflow,
        # we recommend you to use `--validate`
        assert not cfg.omnisource
        if args.validate:
            warnings.warn('val workflow is duplicated with `--validate`, '
                          'it is recommended to use `--validate`. see '
                          'https://github.com/open-mmlab/mmaction2/pull/123')
        val_dataset = copy.deepcopy(cfg.data.val)
        datasets.append(build_dataset(val_dataset))
    if cfg.checkpoint_config is not None:
        # save mmaction version, config file content and class names in
        # checkpoints as meta data
        cfg.checkpoint_config.meta = dict(
            mmaction_version=__version__ + get_git_hash(digits=7),
            config=cfg.pretty_text)

    test_option = dict(test_last=args.test_last, test_best=args.test_best)
    train_model(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=args.validate,
        test=test_option,
        timestamp=timestamp,
        meta=meta)

In [8]:
main(cmd_options)

../Video-Swin-Transformer/configs/_base_/models/swin/swin_tiny.py
../Video-Swin-Transformer/configs/_base_/default_runtime.py


2022-03-13 05:59:43,220 - mmaction - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.6.r11.6/compiler.30794723_0
GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
PyTorch: 1.11.0a0+17540c5
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.3.3 (Git Hash N/A)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_52,code=sm_52;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


2022-03-13 06:00:05,001 - mmaction - INFO - Epoch [1][20/109]	lr: 1.628e-05, eta: 0:51:05, time: 0.943, data_time: 0.182, memory: 2840, top1_acc: 0.1750, top5_acc: 0.3000, loss_cls: 5.6414, loss: 5.6414


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


2022-03-13 06:00:18,141 - mmaction - INFO - Epoch [1][40/109]	lr: 2.288e-05, eta: 0:43:04, time: 0.657, data_time: 0.001, memory: 2843, top1_acc: 0.1750, top5_acc: 0.6500, loss_cls: 5.1258, loss: 5.1258
2022-03-13 06:00:31,363 - mmaction - INFO - Epoch [1][60/109]	lr: 2.949e-05, eta: 0:40:19, time: 0.661, data_time: 0.001, memory: 2843, top1_acc: 0.1750, top5_acc: 0.8750, loss_cls: 3.9770, loss: 3.9770
2022-03-13 06:00:44,641 - mmaction - INFO - Epoch [1][80/109]	lr: 3.609e-05, eta: 0:38:52, time: 0.664, data_time: 0.001, memory: 2843, top1_acc: 0.1250, top5_acc: 1.0000, loss_cls: 2.6258, loss: 2.6258
2022-03-13 06:00:57,978 - mmaction - INFO - Epoch [1][100/109]	lr: 4.270e-05, eta: 0:37:57, time: 0.667, data_time: 0.001, memory: 2843, top1_acc: 0.2250, top5_acc: 1.0000, loss_cls: 1.8580, loss: 1.8580
2022-03-13 06:01:03,729 - mmaction - INFO - Saving checkpoint at 1 epochs
2022-03-13 06:01:21,076 - mmaction - INFO - Epoch [2][20/109]	lr: 5.213e-05, eta: 0:35:57, time: 0.837, data_time

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


2022-03-13 06:24:11,899 - mmaction - INFO - Epoch [19][80/109]	lr: 3.455e-05, eta: 0:13:31, time: 0.682, data_time: 0.000, memory: 2845, top1_acc: 0.1500, top5_acc: 1.0000, loss_cls: 1.6604, loss: 1.6604
2022-03-13 06:24:25,543 - mmaction - INFO - Epoch [19][100/109]	lr: 3.455e-05, eta: 0:13:18, time: 0.682, data_time: 0.000, memory: 2845, top1_acc: 0.2000, top5_acc: 1.0000, loss_cls: 1.6762, loss: 1.6762
2022-03-13 06:24:31,407 - mmaction - INFO - Saving checkpoint at 19 epochs
2022-03-13 06:24:48,495 - mmaction - INFO - Epoch [20][20/109]	lr: 2.966e-05, eta: 0:12:57, time: 0.824, data_time: 0.141, memory: 2845, top1_acc: 0.3000, top5_acc: 1.0000, loss_cls: 1.5742, loss: 1.5742
2022-03-13 06:25:02,136 - mmaction - INFO - Epoch [20][40/109]	lr: 2.966e-05, eta: 0:12:44, time: 0.682, data_time: 0.000, memory: 2845, top1_acc: 0.2250, top5_acc: 1.0000, loss_cls: 1.6653, loss: 1.6653
2022-03-13 06:25:15,774 - mmaction - INFO - Epoch [20][60/109]	lr: 2.966e-05, eta: 0:12:31, time: 0.682, dat